# Algorithm tests
In this notebook, we keep track of algorithm tests. If I am in doubt of whether a certain algorithm is faster or slower, I will test the algorithms using timeit. This will be proof / support for my reasoning about why I make implementation choices.
## Imports

In [6]:
# from Pointcloud.Modules.Object import HelperFunctions

from numpy import arange as np_arange, array as np_array, frompyfunc as np_frompyfunc, max as np_max, ones as np_ones, sort as np_sort
from numpy.random import randint as np_random_randint
from scipy.spatial import KDTree as scipy_spatial_KDTree
from timeit import timeit
from torch import (
    from_numpy as torch_from_numpy,
    rand as torch_rand,
    randperm as torch_randperm,
    zeros as torch_zeros
)
from torch_cluster import knn_graph as torch_cluster_knn_graph

## Triangle area calculation
In the first version of the algorithm I calculated all triangle areas with Numpy, but I found out that IGL also has an implementation. The code below commpares both methods! It turns out IGL is around 92% faster than Numpy, which is a significant speedup!

In [2]:
NUM_TESTS = 100000
algorithm1_str = "algo1"
algorithm2_str = "algo2"

input_arguments = "(example_v, example_f)"
environment = f'''
import numpy as np
from igl import doublearea as igl_doublearea
from numpy import array as np_array, cross as np_cross
from numpy.linalg import norm as np_linalg_norm

example_v = np_array([[-0.03785068,  0.12783747,  0.00448816],
 [-0.044779,    0.128887,    0.001905  ],
 [-0.06801,     0.151244,    0.037195  ],
 [-0.070454,    0.150585,   -0.043458  ],
 [-0.031026,    0.153728,   -0.003546  ],
 [-0.040044,    0.15362,    -0.008167  ]])
example_f = np_array([[2, 5, 3],
 [2, 0, 3],
 [3, 3, 5],
 [3, 5, 3],
 [2, 1, 2],
 [4, 1, 3]])
 
def {algorithm1_str}(v, f):
    triangles = v[f]
    As = triangles[...,1,:] - triangles[...,0,:]
    Bs = triangles[...,2,:] - triangles[...,0,:]
    cross = np_cross(As, Bs, axis=-1)
    areas = 0.5*np_linalg_norm(cross, axis=-1)
    return areas

def {algorithm2_str}(v, f):
    return igl_doublearea(v, f) / 2.0
'''
stmt1 = f"{algorithm1_str}{input_arguments}"
stmt2 = f"{algorithm2_str}{input_arguments}"
speed1 = timeit(stmt=stmt1, setup=environment, number=NUM_TESTS)
speed2 = timeit(stmt=stmt2, setup=environment, number=NUM_TESTS)
print(speed2 / speed1)

0.059847648368175424


## KNN graph calculations
Calculating the KNN graph can be done with Torch and SciPy. Both give back a different representation of the calculation, but what we want to know is which one is faster in the long run (not counting data structures). With the test below, it can be seen that SciPy is around 70% with the predetermined KDTree datastructure. (Tested for k=2 and k=6 for example vertices)

In [3]:
NUM_TESTS = 100000
algorithm1_str = "algo1"
algorithm2_str = "algo2"

input_arguments1 = "(example_v, k)"
input_arguments2 = "(torch_v, k)"
environment = f'''
from numpy import array as np_array
from scipy.spatial import KDTree as scipy_spatial_KDTree
from torch import from_numpy as torch_from_numpy
from torch_cluster import knn_graph

example_v = np_array([[-0.03785068,  0.12783747,  0.00448816],
 [-0.044779,    0.128887,    0.001905  ],
 [-0.06801,     0.151244,    0.037195  ],
 [-0.070454,    0.150585,   -0.043458  ],
 [-0.031026,    0.153728,   -0.003546  ],
 [-0.040044,    0.15362,    -0.008167  ]])
k = 2
torch_v = torch_from_numpy(example_v)

kdtree = scipy_spatial_KDTree(example_v)
 
def {algorithm1_str}(v, k):
    return kdtree.query(v, k)

def {algorithm2_str}(v, k):
    return knn_graph(v, k)
'''
stmt1 = f"{algorithm1_str}{input_arguments1}"
stmt2 = f"{algorithm2_str}{input_arguments2}"
speed1 = timeit(stmt=stmt1, setup=environment, number=NUM_TESTS)
speed2 = timeit(stmt=stmt2, setup=environment, number=NUM_TESTS)
print(speed1 / speed2)

0.32600043832805986


## HelperFunctions.toEdgeTensor works
This section is here just to check whether the functionality of toEdgeTensor works. We compare the result with knn_graph from the torch.cluster package. If the edges are the same in both tensors, the conversion is done correctly.
### Result
It can be seen that the results have almost the same size. The difference is that the scipy variant contains self-loops.

In [4]:
example_v = np_array([[-0.03785068,  0.12783747,  0.00448816],
 [-0.044779,    0.128887,    0.001905  ],
 [-0.06801,     0.151244,    0.037195  ],
 [-0.070454,    0.150585,   -0.043458  ],
 [-0.031026,    0.153728,   -0.003546  ],
 [-0.040044,    0.15362,    -0.008167  ]])
k = 3
original = torch_cluster_knn_graph(torch_from_numpy(example_v), k=k)
kdtree = scipy_spatial_KDTree(example_v)
result = kdtree.query(example_v, k=k+1)[1][:, 1:]
print(original.size(), result.shape)
conversion = HelperFunctions.toEdgeTensor(result)
print(conversion.size(), original.size())
print(conversion)
print(original)


torch.Size([2, 18]) (6, 3)
torch.Size([2, 18]) torch.Size([2, 18])
tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5],
        [1, 4, 5, 0, 5, 4, 1, 0, 5, 5, 4, 1, 5, 0, 1, 4, 1, 0]])
tensor([[1, 4, 5, 0, 5, 4, 1, 0, 5, 5, 4, 1, 5, 0, 1, 4, 1, 0],
        [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]])


## Comparing Uniques
Pandas' unique function seems to be faster than Numpy's unique function. Panda only doesn't sort.. But even with sorting it seems to be faster. We also compare Torch's unique, since we wanna compare all uniques!
### Results (old)
As a clear result, we can see that Torch or Numpy should be used for unique calculations. Not Pandas! Also, you should not convert from one or the other just to use the unique functionality. This is also slower!
### New Results!
Scrap everything! Never use unique anymore if you only need the uniques! Use sets!!!
### Final conclusions!
 ---> Always use sets to calculate uniques, but not if you need counts or inverses. Then use uniques. Also! When using Torch, use Torch's unique and not sets, because conversions take too much time.

In [5]:
NUM_TESTS = 100000
algorithm1_str = "algo1"
algorithm2_str = "algo2"
algorithm3_str = "algo2asdf"
algorithm4_str = "algkasjdf"
algorithm5_str = "algkasadfsadfsadfsjdf"

input_arguments1 = "(numpy_v)"
input_arguments2 = "(pandas_v)"
input_arguments3 = "(torch_v)"
input_arguments4 = "(numpy_v)"
input_arguments5 = "(torch_v)"
environment = f'''
from numpy import array as np_array, unique as np_unique
from numpy.random import randint as np_random_randint
from pandas import Series as pd_Series, unique as pandas_unique
from torch import from_numpy as torch_from_numpy, unique as torch_unique, Tensor as torch_tensor

numpy_v = np_random_randint(10, 20, 10)
torch_v = torch_from_numpy(numpy_v)
pandas_v = pd_Series(numpy_v)
 
def {algorithm1_str}(v):
    return np_unique(v)

def {algorithm2_str}(v):
    return pandas_unique(v)
    
def {algorithm3_str}(v):
    return torch_unique(v)

def {algorithm4_str}(v):
    return np_array(list(set(v.tolist())))

def {algorithm5_str}(v):
    return torch_tensor(list(set(v.tolist()))).int()
'''
stmt1 = f"{algorithm1_str}{input_arguments1}"
stmt2 = f"{algorithm2_str}{input_arguments2}"
stmt3 = f"{algorithm3_str}{input_arguments3}"
stmt4 = f"{algorithm4_str}{input_arguments4}"
stmt5 = f"{algorithm5_str}{input_arguments5}"
speed1 = timeit(stmt=stmt1, setup=environment, number=NUM_TESTS)
speed2 = timeit(stmt=stmt2, setup=environment, number=NUM_TESTS)
speed3 = timeit(stmt=stmt3, setup=environment, number=NUM_TESTS)
speed4 = timeit(stmt=stmt4, setup=environment, number=NUM_TESTS)
speed5 = timeit(stmt=stmt5, setup=environment, number=NUM_TESTS)
results = np_array([speed1, speed2, speed3, speed4, speed5])
print(f"Numpy: {results[0]}\nPandas: {results[1]}\nTorch: {results[2]}\nPython Set Arrays: {results[3]}\nPython Set Tensors: {results[4]}")

Numpy: 0.5249346999999993
Pandas: 1.8982664999999983
Torch: 0.5124120000000012
Python Set Arrays: 0.12064409999999981
Python Set Tensors: 0.8196423999999993


In [6]:
from numpy import array as np_array, unique as np_unique
from numpy.random import randint as np_random_randint
from pandas import Series as pd_Series, unique as pandas_unique
from torch import from_numpy as torch_from_numpy, Tensor as torch_tensor, unique as torch_unique

numpy_v = np_random_randint(10, 20, 10)
torch_v = torch_from_numpy(numpy_v)
pandas_v = pd_Series(numpy_v)
 
def algorithm1_str(v):
    return np_unique(v)

def algorithm2_str(v):
    return pandas_unique(v)
    
def algorithm3_str(v):
    return torch_unique(v)

def algorithm4_str(v):
    return np_array(list(set(v.tolist())))

def algorithm5_str(v):
    return torch_tensor(list(set(v.tolist()))).int()

print(algorithm1_str(numpy_v))
print(algorithm2_str(pandas_v))
print(algorithm3_str(torch_v))
print(algorithm4_str(numpy_v))
print(algorithm5_str(torch_v))

[10 11 12 13 15 18]
[10 15 18 11 12 13]
tensor([10, 11, 12, 13, 15, 18], dtype=torch.int32)
[10 11 12 13 15 18]
tensor([10, 11, 12, 13, 15, 18], dtype=torch.int32)


## FromPyFunc or CumSum
When having multiple ranges and wanting all the unique indices that fall in one of the ranges, there are two methods to compute these values now. When comparing, using the new method stolen from stackoverflow is faster than the old version where FromPyFunc was used.

In [7]:
NUM_TESTS = 100000
algorithm1_str = "algo1"
algorithm2_str = "algo2"

input_arguments1 = "(starts, stops)"
input_arguments2 = "(starts, stops)"
environment = f'''
from numpy import arange as np_arange, array as np_array, frompyfunc as np_frompyfunc, ones as np_ones
from numpy.random import randint as np_random_randint

starts = np_random_randint(10, 20, 100)
stops = np_random_randint(30, 40, 100)

 
def {algorithm1_str}(starts, stops):
    l = stops - starts
    clens = l.cumsum()
    ids = np_ones(clens[-1],dtype=int)
    ids[0] = starts[0]
    ids[clens[:-1]] = starts[1:] - stops[:-1]+1
    return np_array(list(set(ids.cumsum())))

def {algorithm2_str}(starts, stops):
    ranges = np_frompyfunc(np_arange, 2, 1)(starts, stops)
    uniqueFaces = set()
    for range in ranges:
        uniqueFaces.update(range)
    return np_array(list(uniqueFaces))
'''
stmt1 = f"{algorithm1_str}{input_arguments1}"
stmt2 = f"{algorithm2_str}{input_arguments2}"
speed1 = timeit(stmt=stmt1, setup=environment, number=NUM_TESTS)
speed2 = timeit(stmt=stmt2, setup=environment, number=NUM_TESTS)
results = np_array([speed1, speed2])
print(f"boundaries2indices: {results[0]}\nold frompyfunc: {results[1]}")

boundaries2indices: 10.733755299999999
old frompyfunc: 19.1546478


In [8]:
data = np_random_randint(0, 2000, 100).reshape(2, -1)
data[1] = data[0] + 5

 
def algorithm1_str(starts, stops):
    l = stops - starts
    clens = l.cumsum()
    ids = np_ones(clens[-1],dtype=int)
    ids[0] = starts[0]
    ids[clens[:-1]] = starts[1:] - stops[:-1]+1
    return np_array(list(set(ids.cumsum())))

def algorithm2_str(starts, stops):
    ranges = np_frompyfunc(np_arange, 2, 1)(starts, stops)
    uniqueFaces = set()
    for range in ranges:
        uniqueFaces.update(range)
    return np_array(list(uniqueFaces))

print(min(data[0]), max(data[1]))
print(np_sort(algorithm1_str(data[0], data[1])))
print(np_sort(algorithm2_str(data[0], data[1])))

24 1868
[  24   25   26   27   28   29   30   31   32   53   54   55   56   57
   75   76   77   78   79  101  102  103  104  105  159  160  161  162
  163  180  181  182  183  184  221  222  223  224  225  235  236  237
  238  239  297  298  299  300  301  318  319  320  321  322  326  327
  328  329  330  349  350  351  352  353  369  370  371  372  373  462
  463  464  465  466  467  468  469  470  471  579  580  581  582  583
  584  585  586  587  588  608  609  610  611  612  670  671  672  673
  674  730  731  732  733  734  771  772  773  774  775  777  778  779
  780  781  782  799  800  801  802  803  840  841  842  843  844  845
  846  847  848  849  921  922  923  924  925  943  944  945  946  947
  960  961  962  963  964 1044 1045 1046 1047 1048 1104 1105 1106 1107
 1108 1181 1182 1183 1184 1185 1295 1296 1297 1298 1299 1362 1363 1364
 1365 1366 1370 1371 1372 1373 1374 1377 1378 1379 1380 1381 1440 1441
 1442 1443 1444 1502 1503 1504 1505 1506 1551 1552 1553 1554 1555 156

# Matrix 
In the code below, we manually check if a matrix is reversable or not.
For this we use 3 checks:
- The first check checks if there exists a row vector with zeros only.
- The second check checks if there exists a column vector with zeros only.
- The third check checks if there are at least 2 row vectors with at least 2 zero entries and if there are at least 2 column vectors with at least 2 zero entries.

If one of the contidions is met, the matrix is not inversible.

In [8]:
matrix = torch_rand((3, 3))
positions = torch_zeros((9,), dtype=bool)
positions[torch_randperm(9)[:4]] = True
matrix[positions.reshape((3,3))] = 0
print(f"Matrix:\n{matrix}")
print(f"Check 1: {(matrix.sum(dim=0) == 0).any()}\nCheck 2: {(matrix.sum(dim=1) == 0).any()}\nCheck 3: {((matrix == 0).sum(dim=0) > 1).sum() > 1 and ((matrix == 0).sum(dim=1) > 1).sum() > 1}")
print(f"Inverse:\n{matrix.inverse()}")

Matrix:
tensor([[0.0000, 0.4083, 0.2368],
        [0.9687, 0.5019, 0.2336],
        [0.0000, 0.0000, 0.0000]])
Check 1: False
Check 2: True
Check 3: False


_LinAlgError: torch.linalg.inv: The diagonal element 3 is zero, the inversion could not be completed because the input matrix is singular.